<a href="https://colab.research.google.com/github/4bur3k/GN_labs/blob/main/LR_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Варинат 43

In [1]:
from typing import List
from math import log2, ceil
from random import randrange
import binascii

In [2]:
def count_checksum_crc16(data):

    data = data.encode('utf-8')
    crc = 0xFFFF

    for i in range(len(data)):
        crc = crc ^ (data[i] << 8)
        for j in range(8):
            if crc & 0x8000:
                crc = (crc << 1) ^ 0x1021
            else:
                crc = crc << 1
        crc = crc & 0xFFFF
    return crc

In [3]:
source = '''Либертарианство — совокупность политических философий и движений, поддерживающих свободу как основной принцип[1]. Представители либертарианства стремятся максимизировать политическую свободу и автономию, подчёркивая свободу выбора, добровольное объединение и индивидуальное суждение[2][3][4]. Они разделяют скептицизм по отношению к государственной власти, но расходятся в масштабах своей оппозиции по отношению к существующим экономическим и политическим системам. Различные школы либертарианской мысли предлагают различные взгляды относительно законных функций государственной и частной власти, часто призывая к ограничению или роспуску принудительных социальных институтов[5]. Термин «либертарианец» (англ. libertarian) впервые появился в эссе «О свободе и необходимости» (1789) американского унитарианского философа Уильяма Белшама[en]. Он был производным от «свободы» (англ. liberty) по аналогии с образованием слова «унитарий» (англ. unitarian) от «единства» (англ. unity) в английском языке и обозначал сторонников философской доктрины «свободы воли»[9]. В своём эссе Белшам осудил идеи, которые он приписывал либертарианству и противопоставил им религиозный детерминизм.Во второй половине XIX века термин «либертарий» (фр. la libertaire) в значении «анархист» взяли на вооружение представители французского анархизма. В 1858 году Жозеф Дежак начал издавать в США газету «Le Libertaire, Journal du Mouvement Social». В 1880 году на конгрессе в Гавре анархо-коммунисты также употребили этот термин. Росту его популярности в конце XIX века способствовало введение направленных против анархизма ограничений на свободу печати во Франции после теракта, осуществлённого Огюстом Вальяном в 1893 году. В результате анархисты начали широко использовать фр. libertaire как эвфемизм вместо слова фр. anarchiste, в частности, в 1895 году была учреждена газета «Le Libertaire». Впоследствии «либертарианство» и «анархизм» стали синонимами, также положив начало термину «либертарный социализм».В современном значении термин «либертарианство» впервые начал использовать американский политик и основатель Фонда экономического образования Леонард Рид[en], в 1940-е годы провозгласивший себя либертарианцем. Вслед за ним многие сторонники личной и экономической свободы также стали называть себя либертарианцами, чтобы отличать себя от «либералов», под которыми в США и некоторых других странах начиная с XX века понимают сторонников личной свободы и государственного перераспределения ресурсов (в частности, «нового курса» Рузвельта), включая социал-демократов и умеренных коммунистов. Тем не менее, многие сторонники либертарианских идей не называют себя либертарианцами, настаивая на традиционном обозначении своей идеологии («либерализм») или определяя себя как «классических либералов». Другие считают подобную приверженность старым терминам ошибочной, вносящей путаницу в политическую картину современного мира, что мешает распространению и пониманию либертарианских идей.'''
crc = count_checksum_crc16(source)

In [4]:
print("CRC-16 для сообщения:\n{}\n: {}".format(source, hex(crc)))

CRC-16 для сообщения:
Либертарианство — совокупность политических философий и движений, поддерживающих свободу как основной принцип[1]. Представители либертарианства стремятся максимизировать политическую свободу и автономию, подчёркивая свободу выбора, добровольное объединение и индивидуальное суждение[2][3][4]. Они разделяют скептицизм по отношению к государственной власти, но расходятся в масштабах своей оппозиции по отношению к существующим экономическим и политическим системам. Различные школы либертарианской мысли предлагают различные взгляды относительно законных функций государственной и частной власти, часто призывая к ограничению или роспуску принудительных социальных институтов[5]. Термин «либертарианец» (англ. libertarian) впервые появился в эссе «О свободе и необходимости» (1789) американского унитарианского философа Уильяма Белшама[en]. Он был производным от «свободы» (англ. liberty) по аналогии с образованием слова «унитарий» (англ. unitarian) от «единства» (англ. unity)

In [5]:
def __hamming_common(src: List[List[int]], s_num: int, encode=True) -> int:
    s_range = range(s_num)
    errors = 0
    for i in src:
        sindrome = 0
        for s in s_range:
            sind = 0
            for p in range(2 ** s, len(i) + 1, 2 ** (s + 1)):
                for j in range(2 ** s):
                    if (p + j) > len(i):
                        break
                    sind ^= i[p + j - 1]
            if encode:
                i[2 ** s - 1] = sind
            else:
                sindrome += (2 ** s * sind)
        if (not encode) and sindrome:
            try:
                i[sindrome - 1] = int(not i[sindrome - 1])
            except IndexError:
                errors += 1
    return errors

In [6]:
def hamming_encode(msg: str, mode: int = 8) -> str:
  
    result = str()

    msg_b = msg.encode("utf8")
    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    bit_seq = []
    for byte in msg_b:  # get bytes to binary values; every bits store to sublist
        bit_seq += list(map(int, f"{byte:08b}"))

    res_len = ceil((len(msg_b) * 8) / mode)     # length of result (bytes)
    bit_seq += [0] * (res_len * mode - len(bit_seq))    # filling zeros

    to_hamming = []

    for i in range(res_len):    # insert control bits into specified positions
        code = bit_seq[i * mode:i * mode + mode]
        for j in range(s_num):
            code.insert(2 ** j - 1, 0)
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, True)   # process

    for i in to_hamming:
        result += "".join(map(str, i))

    return result

In [7]:
def hamming_decode(msg: str, mode: int = 8):

    result = ""

    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    res_len = len(msg) // (mode + s_num)    # length of result (bytes)
    code_len = mode + s_num     # length of one code sequence

    to_hamming = []

    for i in range(res_len):    # convert binary-like string to int-list
        code = list(map(int, msg[i * code_len:i * code_len + code_len]))
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, False)  # process

    for i in to_hamming:    # delete control bits
        for j in range(s_num):
            i.pop(2 ** j - 1 - j)
        result += "".join(map(str, i))

    msg_l = []

    for i in range(len(result) // 8):   # convert from binary-sring value to integer
        val = "".join(result[i * 8:i * 8 + 8])
        msg_l.append(int(val, 2))

    # finally decode to a regular string
    try:
        result = bytes(msg_l).decode("utf-8")
    except UnicodeDecodeError:
        pass

    return result, errors

In [8]:
def noizer(msg: str, mode: int) -> str:
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize = randrange(code_len)
        to_noize[noize] = int(not to_noize[noize])
        result += "".join(map(str, to_noize))

    return result

In [9]:
def noizer4(msg: str, mode: int) -> str:
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize1 = randrange(code_len)
        noize2 = randrange(code_len)
        noize3 = randrange(code_len)
        noize4 = randrange(code_len)
        to_noize[noize1] = int(not to_noize[noize1])
        to_noize[noize2] = int(not to_noize[noize2])
        to_noize[noize3] = int(not to_noize[noize3])
        to_noize[noize4] = int(not to_noize[noize4])
        result += "".join(map(str, to_noize))

    return result

In [12]:
MODE = 70
msg = "Либертарианство — совокупность политических философий и движений, поддерживающих свободу как основной принцип[1]. Представители либертарианства стремятся максимизировать политическую свободу и автономию, подчёркивая свободу выбора, добровольное объединение и индивидуальное суждение[2][3][4]. Они разделяют скептицизм по отношению к государственной власти, но расходятся в масштабах своей оппозиции по отношению к существующим экономическим и политическим системам. Различные школы либертарианской мысли предлагают различные взгляды относительно законных функций государственной и частной власти, часто призывая к ограничению или роспуску принудительных социальных институтов[5]. Термин «либертарианец» (англ. libertarian) впервые появился в эссе «О свободе и необходимости» (1789) американского унитарианского философа Уильяма Белшама[en]. Он был производным от «свободы» (англ. liberty) по аналогии с образованием слова «унитарий» (англ. unitarian) от «единства» (англ. unity) в английском языке и обозначал сторонников философской доктрины «свободы воли»[9]. В своём эссе Белшам осудил идеи, которые он приписывал либертарианству и противопоставил им религиозный детерминизм.Во второй половине XIX века термин «либертарий» (фр. la libertaire) в значении «анархист» взяли на вооружение представители французского анархизма. В 1858 году Жозеф Дежак начал издавать в США газету «Le Libertaire, Journal du Mouvement Social». В 1880 году на конгрессе в Гавре анархо-коммунисты также употребили этот термин. Росту его популярности в конце XIX века способствовало введение направленных против анархизма ограничений на свободу печати во Франции после теракта, осуществлённого Огюстом Вальяном в 1893 году. В результате анархисты начали широко использовать фр. libertaire как эвфемизм вместо слова фр. anarchiste, в частности, в 1895 году была учреждена газета «Le Libertaire». Впоследствии «либертарианство» и «анархизм» стали синонимами, также положив начало термину «либертарный социализм».В современном значении термин «либертарианство» впервые начал использовать американский политик и основатель Фонда экономического образования Леонард Рид[en], в 1940-е годы провозгласивший себя либертарианцем. Вслед за ним многие сторонники личной и экономической свободы также стали называть себя либертарианцами, чтобы отличать себя от «либералов», под которыми в США и некоторых других странах начиная с XX века понимают сторонников личной свободы и государственного перераспределения ресурсов (в частности, «нового курса» Рузвельта), включая социал-демократов и умеренных коммунистов. Тем не менее, многие сторонники либертарианских идей не называют себя либертарианцами, настаивая на традиционном обозначении своей идеологии («либерализм») или определяя себя как «классических либералов». Другие считают подобную приверженность старым терминам ошибочной, вносящей путаницу в политическую картину современного мира, что мешает распространению и пониманию либертарианских идей."
print(f'Сообщение:\n{msg}')
checksum = count_checksum_crc16(msg)
print(f'Контрольная сумма: {checksum}')

# Первая отправка (без ошибок)
print('-----------ПЕРВАЯ ОТПРАВКА-----------')
enc_msg = hamming_encode(msg, MODE)
print(f'Кодированное сообщение:\n{enc_msg}')
dec_msg, err = hamming_decode(enc_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {count_checksum_crc16(dec_msg)}, корректность: {count_checksum_crc16(dec_msg) == checksum}')
print(f'MSG: {msg == dec_msg}')

# Вторая отправка (не более 1 ошибки на слово)
print('-----------ВТОРАЯ ОТПРАВКА-----------')
noize_msg = noizer(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {count_checksum_crc16(dec_msg)}, корректность: {count_checksum_crc16(dec_msg) == checksum}')
print(f'MSG: {msg == dec_msg}')

# Третья отправка (4 ошибки на слово)
print('-----------ТРЕТЬЯ ОТПРАВКА-----------')
noize_msg = noizer4(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {count_checksum_crc16(dec_msg)}, корректность: {count_checksum_crc16(dec_msg) == checksum}, количество обнаруженных ошибок: {err}')

Сообщение:
Либертарианство — совокупность политических философий и движений, поддерживающих свободу как основной принцип[1]. Представители либертарианства стремятся максимизировать политическую свободу и автономию, подчёркивая свободу выбора, добровольное объединение и индивидуальное суждение[2][3][4]. Они разделяют скептицизм по отношению к государственной власти, но расходятся в масштабах своей оппозиции по отношению к существующим экономическим и политическим системам. Различные школы либертарианской мысли предлагают различные взгляды относительно законных функций государственной и частной власти, часто призывая к ограничению или роспуску принудительных социальных институтов[5]. Термин «либертарианец» (англ. libertarian) впервые появился в эссе «О свободе и необходимости» (1789) американского унитарианского философа Уильяма Белшама[en]. Он был производным от «свободы» (англ. liberty) по аналогии с образованием слова «унитарий» (англ. unitarian) от «единства» (англ. unity) в английск